# Ⅰ. 주요 개념

##### 협업 필터링(Collaborative Filtering, CF)
다수 사용자의 아이템 구매 이력 정보만으로 사용자 간 유사성 및 아이템 간 유사성을 파악. 아이템과 사용자 간의 행동/관계에 주목    
참고논문 : Collaborative Filtering for Implicit Feedback Datasets http://yifanhu.net/PUB/cf.pdf

##### 콘텐츠 기반 필터링
아이템의 고유 정보를 바탕으로 아이템 간 유사성을 파악. 아이템 고유 속성에 주목
##### 콜드 스타트 cold start
시스템이 충분한 정보를 모으지 못한 사용자나 아이템에 대한 추론을 할 수 없는 상태를 말함. 협업 필터링에서의 주요 제약요인 중 하나다. 이를 극복하는 방안으로 회원가입시 개인정보에서 아이템과 관련된 정보 수집.
##### 암묵적(implicit) 데이터
아이템에 대한 명확한 선호를 제시한 것 외에 다른 정보로 선호를 유추할 수 있는 평가하는 것을 말함. e.g) 음원의 경우 플레이 횟수, 플레이 시간, 클릭 수, 구매 여부, 플레이 스킵 여부, 검색 기록, 방문 페이지 이력, 구매 내역 등
##### MF(Matrix Factorization)
m,n) 사이즈의 행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해한다면 R이란 그저 P와 Q의 행렬곱으로 표현 가능할 수 있다
##### CSR(Compressed Sparse Row) Matrix
행렬의 크기가 매우 크면 이를 저장하고 연산해야 할 메모리의 제약으로, 제한된 메모리에 용량이 큰 행렬을 담을 수 있게 하는 기법. 유효한 값을 가진 데이터의 값과 좌표 정보만 구성함으로써 메모리 사용량을 최소화함.

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

row = np.array([0, 0, 1, 2, 2, 2])
col = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6])

a = csr_matrix((data, (row, col)), shape=(3, 3)).toarray()
a

# Ⅱ. 프로세스와 주요 함수/클래스

## 1. 프로세스    
#### (1) MF Matric 만들기    
    - (사용자*잠재적 요인)행렬과 (잠재적 요인*영화)행렬 생성   
    - 사용자별 영화에 대한 시청횟수가 모두 있는게 아니므로 CSR Matrix로 생성     
#### (2) implict의 als모델로 학습    
     - 여러 특성을 한꺼번에 학습할 수 없으므로 한쪽을 고정시키고 학습하고 교대하여 학습하는 알고리즘  
     - 학습후 각 특성변수의 벡터가 생성되고 이것으로 유사한 선호 아이템을 추천할 수 있다.     
     - 추천 아이템에 대한 기여도로 추천 이유에 대해서 이해할 수 있다.

## 2. 패키지의 함수/클래스

- pandas.DataFrame.unique()은 특정 컬럼에 포함된 유니크한 데이터만 모아 줍니다. indexing 작업을 위해 매우 유용   
- pandas.DataFrame.nunique()은 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용
  
  
##### implicit 패키지
- als(AlternatingLeastSquares) 모델 : Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행   
- AlternatingLeastSquares 클래스의 __init__ 파라미터를 살펴보겠습니다.   
  ① factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지     
  ② regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지    
  ③ use_gpu : GPU를 사용할 것인지     
  ④ iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지    
- AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 아티스트를 찾습니다   
- AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 제가 좋아할 만한 아티스트를 추천받는다. (filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument)    
- AlternatingLeastSquares 클래스에 구현된 explain 메소드를 사용하면 제가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있습니다

# Ⅲ. 코드(영화 추천)

In [ ]:
# !mkdir -p ~/aiffel/Exploration/09_recommendata_iu/data/ml-1m
# !ln -s ~/data/ml-1m/* ~/aiffel/Exploration/09_recommendata_iu/data/ml-1m

영화관련 추천시스템의 MNIST라고 부를만한 Movielens 데이터를 사용하기로 한다.   
유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있다. MovieLens 1M Dataset 사용하겠다.

## 1. 데이터 준비와 전처리

In [1]:
import os
import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/Exploration/09_recommendation_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/Exploration/09_recommendation_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
#  movies테이블에서 영화관련 정보를 ratings테이블로 가져온다.(pd.merge())
df = pd.merge(ratings, movies, on='movie_id')
df[:5]

,user_id,movie_id,ratings,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama


In [5]:
# 검색을 쉽게하기 위해 영화 제목과 장르 문자열을 소문자로 바꿔줍시다.
df['title'] = df['title'].str.lower() 
df['genre'] = df['genre'].str.lower() 

# ratings 컬럼의 이름을 counts로 바꿉니다.
df.rename(columns={'ratings':'counts'}, inplace=True)

df.tail(10)

,user_id,movie_id,counts,timestamp,title,genre
1000199,5334,3382,5,960796159,song of freedom (1936),drama
1000200,5420,1843,3,960156505,slappy and the stinkers (1998),children's|comedy
1000201,5433,286,3,960240881,nemesis 2: nebula (1995),action|sci-fi|thriller
1000202,5494,3530,4,959816296,smoking/no smoking (1993),comedy
1000203,5556,2198,3,959445515,modulations (1998),documentary
1000204,5949,2198,5,958846401,modulations (1998),documentary
1000205,5675,2703,3,976029116,broken vessels (1998),drama
1000206,5780,2845,1,958153068,white boys (1999),drama
1000207,5851,3607,5,957756608,one little indian (1973),comedy|drama|western
1000208,5938,2909,4,957273353,"five wives, three secretaries and me (1998)",documentary


In [6]:
df = df[['user_id', 'title', 'counts']]
df.tail()

,user_id,title,counts
1000204,5949,modulations (1998),5
1000205,5675,broken vessels (1998),3
1000206,5780,white boys (1999),1
1000207,5851,one little indian (1973),5
1000208,5938,"five wives, three secretaries and me (1998)",4


### 사용자의 암묵적(implicit) 정보

- 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트하겠다.
- 별점을 시청횟수로 해석해서 생각한다.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외한다.

In [7]:
# 3점 이상만 남깁니다.
df = df[df['counts']>=3]
filtered_data_size = len(df)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [8]:
#  첫 번째 유저가 어떤 영화를 보는지 확인
condition_u = (ratings['user_id']== ratings.loc[0, 'user_id'])
ratings.loc[condition_u]

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


## 2. 데이터 분석 및 전처리

### 데이터 분석

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836478 entries, 0 to 1000208
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  836478 non-null  int64 
 1   title    836478 non-null  object
 2   counts   836478 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 25.5+ MB


In [10]:
# 사용자 수
num_user = df['user_id'].nunique()
# 영화 수
num_movie = df['title'].nunique()

print('사용자 수 : ', num_user, '         영화 수 : ', num_movie)

사용자 수 :  6039          영화 수 :  3628


In [11]:
# 인기 많은 영화 30개(인기순)
movie_count = df.groupby('title')['counts'].sum()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   14449
star wars: episode iv - a new hope (1977)                13178
star wars: episode v - the empire strikes back (1980)    12648
saving private ryan (1998)                               11348
star wars: episode vi - return of the jedi (1983)        11303
raiders of the lost ark (1981)                           11179
silence of the lambs, the (1991)                         11096
matrix, the (1999)                                       10903
sixth sense, the (1999)                                  10703
terminator 2: judgment day (1991)                        10513
fargo (1996)                                             10465
schindler's list (1993)                                  10317
braveheart (1995)                                        10125
shawshank redemption, the (1994)                         10085
back to the future (1985)                                10081
godfather, the (1972)                            

In [12]:
# 유저별 관람한 영화 갯수 통계
user_count = df.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

In [13]:
# 유저별 영화 관람횟수 중앙값에 대한 통계
user_median = df.groupby('user_id')['counts'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

# 3. 내가 선호하는 영화를 5가지 골라서 DF에 추가

In [14]:
# 내가 선호하는 영화 5개를 추가
# 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['braveheart (1995)' , 'shawshank redemption, the (1994)' ,'forrest gump (1994)' ,
               'silence of the lambs, the (1991)' ,'raiders of the lost ark (1981)']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['lee']*5, 'title': my_favorite, 'counts':[5]*5})

if not df.isin({'user_id':['lee']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    df = df.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

df.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,title,counts
1000203,5556,modulations (1998),3
1000204,5949,modulations (1998),5
1000205,5675,broken vessels (1998),3
1000207,5851,one little indian (1973),5
1000208,5938,"five wives, three secretaries and me (1998)",4
0,lee,braveheart (1995),5
1,lee,"shawshank redemption, the (1994)",5
2,lee,forrest gump (1994),5
3,lee,"silence of the lambs, the (1991)",5
4,lee,raiders of the lost ark (1981),5


# 4. CSR Matrix 만들기

## (1) 모델에 활용하기 위한 전처리

In [15]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = df['user_id'].unique()
movie_unique = df['title'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [16]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['lee'])    # 6039명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(movie_to_idx['braveheart (1995)'])

6039
92


In [17]:
'''
indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
'''

df['user'] = df['user_id'].map(user_to_idx)
df['movie'] = df['title'].map(movie_to_idx)
df.tail(10)

,user_id,title,counts,user,movie
1000203,5556,modulations (1998),3,1532,3624
1000204,5949,modulations (1998),5,1649,3624
1000205,5675,broken vessels (1998),3,1564,3625
1000207,5851,one little indian (1973),5,1621,3626
1000208,5938,"five wives, three secretaries and me (1998)",4,4159,3627
0,lee,braveheart (1995),5,6039,92
1,lee,"shawshank redemption, the (1994)",5,6039,167
2,lee,forrest gump (1994),5,6039,171
3,lee,"silence of the lambs, the (1991)",5,6039,128
4,lee,raiders of the lost ark (1981),5,6039,127


In [18]:
num2_user = len(user_to_idx)
num2_movie = len(movie_to_idx)
print('유저 수 : ', num2_user, '영화 수 : ', num2_movie)

유저 수 :  6040 영화 수 :  3628


In [19]:
# csr matrix 생성을 위한 행과 열의 특성변수 추출
csr_row = df['user']
csr_col = df['movie']
print(csr_row); print(csr_col)

0       0
1       1
2       2
3       3
4       4
     ... 
0    6039
1    6039
2    6039
3    6039
4    6039
Name: user, Length: 836483, dtype: int64
0      0
1      0
2      0
3      0
4      0
    ... 
0     92
1    167
2    171
3    128
4    127
Name: movie, Length: 836483, dtype: int64


In [20]:
# csr matrix 생성을 위한 값 추출/저장
df['counts'] = df['counts'].values.astype(np.float32)

## (2) CSR(Compressed Sparse Row) Matrix 만들기

In [21]:
# csr matrix 생성
num3_user = df['user'].nunique()
num3_movie = df['movie'].nunique()

csr_row = df['user']
csr_col = df['movie']

csr_data = csr_matrix((df.counts, (csr_row, csr_col)),
                      shape= (num3_user, num3_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.float32'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5. AlternatingLeastSquares 모델 구성/학습

In [22]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

### 모델 설정

In [23]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, 
                                    iterations=15, dtype=np.float32)

In [24]:
# als 모델은 input으로 (item X user 꼴의 matrix)를 받기 때문에 Transpose해줍니다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.float32'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [25]:
# 모델 학습
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

# 6. 나의 선호도 파악

In [26]:
lee, braveheart = user_to_idx['lee'], movie_to_idx['braveheart (1995)']
lee_vector, braveheart_vector = als_model.user_factors[lee], als_model.item_factors[braveheart]

print('슝=3')

슝=3


In [27]:
lee_vector

array([-4.7407821e-01, -3.0173787e-01, -5.5350411e-01, -1.2507228e+00,
        1.3797371e-01, -3.4291393e-01,  8.8551545e-01,  9.9414104e-01,
        1.8046124e-01, -3.0430961e-01, -2.0363148e-01,  3.5292305e-02,
       -2.8158364e-01,  3.5453761e-01, -2.1244992e-01, -4.1135246e-01,
        3.7123090e-01,  3.6935526e-01,  2.9843956e-01,  4.8766392e-01,
       -3.5362867e-01, -1.2296803e-01,  9.3423569e-01,  9.7524828e-01,
        7.1684283e-01, -5.7151508e-01, -4.8107657e-01,  3.3009452e-01,
       -2.4023756e-01,  8.9326531e-02, -7.7849162e-01,  8.0301680e-02,
        4.9506012e-01,  4.3592051e-01,  1.8988363e-01, -8.0260806e-02,
        8.8926011e-01, -1.1861677e-01,  2.1310141e-02, -3.6794779e-01,
       -1.8095101e-01,  2.2329676e-01,  8.9875442e-01,  4.9379337e-01,
       -3.9011055e-01, -5.0211579e-01,  3.6564910e-01,  1.2083043e+00,
        4.7940263e-01,  3.1354675e-01, -2.5169516e-01, -1.0373830e+00,
       -2.2473115e-01,  3.3978261e-02, -7.5652272e-01,  2.5914901e-01,
      

In [28]:
braveheart_vector

array([-0.01363069,  0.00192601, -0.00441781,  0.00404347, -0.0131976 ,
        0.04178271,  0.02518606, -0.00578348, -0.0282063 , -0.02094601,
       -0.03096948,  0.00757498,  0.00216159,  0.01567119, -0.00685607,
       -0.00342591, -0.00818138,  0.00024078,  0.00023421,  0.01016039,
        0.00679173,  0.00160818,  0.01430951,  0.03901727,  0.03974763,
       -0.02048182,  0.00810888,  0.02640343, -0.0291489 , -0.00542768,
       -0.00721166,  0.0162219 ,  0.02383581, -0.00832243,  0.00140947,
       -0.01470398,  0.03068289, -0.00241532,  0.01778409, -0.01596903,
        0.02423947, -0.00172261,  0.04022402, -0.00809222,  0.00994227,
        0.00892343,  0.00298762,  0.02835308, -0.000143  ,  0.0101369 ,
       -0.02108472, -0.03101372, -0.02219284,  0.01676557,  0.01454677,
       -0.00963585,  0.00493853, -0.02798849,  0.01082167, -0.00166858,
       -0.00261574, -0.00040846, -0.00090946,  0.00522209, -0.0153891 ,
       -0.00172048,  0.01972117,  0.03813114,  0.01446379, -0.02

In [29]:
# lee와 braveheart를 내적하는 코드
np.dot(lee_vector, braveheart_vector)

0.65486914

나의 영화에 대한 선호도를 어떻게 예측하는지 확인하자.  
사용자와 아티스트 벡터 내적이 0.5를 기준으로 그보다 높으면 모델이 선호한다고 판단할 수 있다.  그런 관점에서 "브레이브 하트"는 내가 좋아하는 영화중 하나로 수긍이 가는 점수를 제시한다.

In [30]:
title_id = movie_to_idx['fargo (1996)']   
movie_vector = als_model.item_factors[title_id]
np.dot(lee_vector, movie_vector)

0.39180237

모델은 내가 '파고'를 그다지 좋아하지 않을 것으로 보내요.

# 7. 비슷한 영화 찾기 : simmilar_items 메서드 사용

AlternatingLeastSquares 클래스에 구현되어 있는 similar_items 메서드를 통하여 비슷한 아티스트를 찾습니다. 처음으로는 제가 좋아하는 "양들의 침묵"으로 찾아보겠습니다.

In [31]:

def get_similar_movie(title: str):
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    
    movie_id = movie_to_idx[title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [32]:
get_similar_movie('silence of the lambs, the (1991)')

['silence of the lambs, the (1991)',
 'shawshank redemption, the (1994)',
 'fargo (1996)',
 'pulp fiction (1994)',
 "schindler's list (1993)",
 'usual suspects, the (1995)',
 'goodfellas (1990)',
 'good will hunting (1997)',
 'sixth sense, the (1999)',
 'sling blade (1996)']

유사한 영화로 '쇼생크 탈출', '파고', '퍼플 픽션', '쉰들러 리스트', '좋은 친구들', '언유졀 서스펙트', '식스 센스', '굿윌헌팅' '엘에이 컨피던셜'을 제시하였다.   
'세븐'이 들어가지 않은게 아쉽고, '파고'는 왜 들어갔는지 의문이다.

# 8. 유저에게 영화 추천하기 : recommend와 explain 메서드

In [38]:
def get_recommanded_movie(user: str):
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    
    user = user_to_idx['lee']
    movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
    movie_recommended = [idx_to_movie[i[0]] for i in movie_recommended]
    return movie_recommended


def get_explained_movie(user:str, title:str):
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    
    user = user_to_idx['lee']
    title_id = movie_to_idx[title]
    explain = als_model.explain(user, csr_data, itemid=title_id)
    explain = [(idx_to_movie[i[0]], i[1]) for i in explain[1]]
    return explain

AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드를 통하여 제가 좋아할 만한 아티스트를 추천받습니다. filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [39]:
get_recommanded_movie('lee')

['saving private ryan (1998)',
 "schindler's list (1993)",
 'star wars: episode v - the empire strikes back (1980)',
 'star wars: episode iv - a new hope (1977)',
 'fargo (1996)',
 'good will hunting (1997)',
 'star wars: episode vi - return of the jedi (1983)',
 'pulp fiction (1994)',
 'american beauty (1999)',
 'princess bride, the (1987)',
 'groundhog day (1993)',
 'back to the future (1985)',
 'boat, the (das boot) (1981)',
 'fugitive, the (1993)',
 'goodfellas (1990)',
 'l.a. confidential (1997)',
 'indiana jones and the last crusade (1989)',
 'sixth sense, the (1999)',
 'matrix, the (1999)',
 'die hard (1988)']

### 추천영화 추천 이유

모델이 영화를 추천해준 이유를 알기 위해 AlternatingLeastSquares 클래스에 구현된 explain 메소드를 사용한다.   나가 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있습니다.  어떤 영화들이 추천점수에 얼마나 기여하고 있는걸까?

In [40]:
get_explained_movie('lee','good will hunting (1997)' )

[('shawshank redemption, the (1994)', 0.16981466274129955),
 ('silence of the lambs, the (1991)', 0.09780844883955331),
 ('braveheart (1995)', 0.060877245022735206),
 ('raiders of the lost ark (1981)', 0.030128754619611026),
 ('forrest gump (1994)', 0.019467321539716337)]

쇼생크탈출 > 양들의 침묵 > 브레이브하트 > 레이더스 잃어버린 성괘 > 포레스트 검프 순으로 추천에 영향을 주었다.

# Ⅳ. 결론

## 1. 결과 분석

- 유사한 영화목록은 대체로 무난하지만 명확한 이유는 알 수 없다.   
- 추천 영화 목록은 대부분 내가 봤던 영화였고 좋아하는 영화들이다.   
- 추천이유도 대체로 타당해 보인다.

## 2. 차후 과제

- 상품목록과 상품의 특성을 다양한 특성요소로 반영하여 학습하고 결과를 도출
- 본 과정을 진행하면서 조금 아쉬운 것은 MF 모델과 ALS모델이 혼재되었다는 점이다. 같은 것인듯 다른 것인듯하다.    
- 다양한 추천 알고리즘이 있다고 한다. 다양한 모델을 비교 해보자.